# Processing the BlastP results

In [ ]:
%run ../config/init.py
import itertools
from goenrichment.go import parse_go_obo

### Loading Taxonomy network

In [ ]:
pickle_file = os.path.join(DATA, 'taxonomy', 'taxonomy_networkx.pickle')
tax = pickle.load(open(pickle_file, "rb"))
viridiplantae = [int(i) for i in successors('33090', tax)]  
print('{} taxonomies IDs in the list'.format(len(viridiplantae)))

### Loading GO network

In [ ]:
#Downloading gene_info
if not os.path.exists(os.path.join(DATA, 'go-basic.obo')):
    os.chdir(DATA)
    !wget http://current.geneontology.org/ontology/go-basic.obo
go = nx.DiGraph()
entries = parse_go_obo(os.path.join(DATA, 'go-basic.obo'))
nodes, edges = zip(*entries)
go.add_nodes_from(nodes)
go.add_edges_from(itertools.chain.from_iterable(edges))
go.graph['roots'] = {data['name']: n for n, data in go._node.items() \
                     if 'name' in data and data['name'] == data['namespace']}


### Loading flat databases for cross-referencing

In [ ]:
# Download the UniProt ID mapping to link GO with Protein GI
if not os.path.exists(os.path.join(DATA, 'idmapping_selected.pkl')):
    if not os.path.exists(os.path.join(DATA, 'idmapping_selected.tab.gz')):
        os.chdir(DATA)
        !wget ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/idmapping_selected.tab.gz

    # Total number of lines while writing this code. Use it as a rough estimate 
    total_lines = 181252700 
    bar_length = 100
    idmapping = pandas.DataFrame()
    total = 0
    for chunk in pandas.read_csv(os.path.join(DATA, 'idmapping_selected.tab.gz'), header=None, sep='\t', chunksize=10000, low_memory=False):    
        total += len(chunk)
        progress = total * 100/total_lines
        i = int(progress)
        chunk = chunk[(chunk[12].isin(viridiplantae)) & (chunk[2].notnull()) & (chunk[4].notnull())]        
        if not chunk.empty:
            chunk = chunk[[2, 3, 6, 7, 8, 9, 12]]
            chunk = chunk.drop_duplicates()
            idmapping = pandas.concat([idmapping, chunk])
        print('{0}/{1} [{2}] {3:.1f}%'.format(len(idmapping), total, "#" * i + "-" * (bar_length - i), progress), end='\r')
    del chunk
    print()
    idmapping = idmapping.rename(columns={2:'GeneID', 3:'PAs', 6:'GOES', 7:'UniRef100', 8:'UniRef90', 9:'UniRef50', 12:'#tax_id'})
    idmapping = idmapping.assign(PAs=idmapping.PAs.astype(str))
    idmapping = idmapping.assign(GOES=idmapping.GOES.astype(str))
    df = pandas.DataFrame(idmapping.PAs.str.split(';').tolist(), index=idmapping.GeneID).stack()
    df = df.reset_index([0, 'GeneID'])
    df = df.rename(columns={0:'protein_accession.version'})
    idmapping = idmapping.merge(df, on='GeneID')    
    new = idmapping['protein_accession.version'].str.split(".", n = 1, expand = True) 
    idmapping["protein_accession"] = new[0]
    idmapping = idmapping.drop(['PAs', 'protein_accession.version'], axis=1)
    df = pandas.DataFrame(idmapping.GOES.str.split(';').tolist(), index=idmapping.GeneID).stack()
    df = df.reset_index([0, 'GeneID'])
    df = df.rename(columns={0:'GO'})
    idmapping = idmapping.merge(df, on='GeneID')
    idmapping = idmapping.drop(['GOES'], axis=1)
    idmapping = idmapping[idmapping['GO'].str.startswith('GO:')]       
    idmapping = idmapping.drop_duplicates()    
    idmapping = idmapping.reset_index(drop=True)
    del df
    idmapping.to_pickle(os.path.join(DATA, 'idmapping_selected.pkl'))
else:
    idmapping = pandas.read_pickle(os.path.join(DATA, 'idmapping_selected.pkl'))
    print('idmapping: {}'.format(len(idmapping)))

display(idmapping.head())

In [ ]:
# Downloading ec2go map
if not os.path.exists(os.path.join(DATA, 'ec2go')):
    os.chdir(DATA)
    !wget http://geneontology.org/external2go/ec2go
ec2go = pandas.read_csv(os.path.join(DATA, 'ec2go'), sep=' \> ', comment='!', header=None, engine='python')
new = ec2go[1].str.split(" ; ", n = 1, expand = True)
ec2go[1] = new[1]
ec2go.head()

In [ ]:
#Downloading gene2go
if not os.path.exists(os.path.join(DATA, 'gene2go.gz')):
    os.chdir(DATA)
    !wget https://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
gene2go = pandas.read_csv(os.path.join(DATA, 'gene2go.gz'), sep='\t')
t = len(gene2go)
gene2go = gene2go[gene2go['#tax_id'].isin(viridiplantae)]
f = len(gene2go)
print('{}/{} gene2go terms in viridiplantae'.format(f,t))

display(gene2go)

In [ ]:
if not os.path.exists(os.path.join(DATA, 'gene2accession.pkl')):
    if not os.path.exists(os.path.join(DATA, 'gene2accession.gz')):
        os.chdir(DATA)
        !wget https://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2accession.gz

    gene2accession = pandas.DataFrame()
    for chunk in pandas.read_csv(os.path.join(DATA, 'gene2accession.gz'), sep='\t', chunksize=10000, low_memory=False):    
        chunk = chunk[chunk['#tax_id'].isin(viridiplantae)]
        if not chunk.empty:
            chunk = chunk[['#tax_id', 'GeneID', 'protein_accession.version','protein_gi', 'genomic_nucleotide_accession.version','genomic_nucleotide_gi']]
            chunk = chunk.drop_duplicates()
            gene2accession = pandas.concat([gene2accession, chunk])
            print('gene2accession: {}'.format(len(gene2accession)), end='\r')
    del chunk    
    print()
    gene2accession.to_pickle(os.path.join(DATA, 'gene2accession.pkl'))
else:
    gene2accession = pandas.read_pickle(os.path.join(DATA, 'gene2accession.pkl'))
    print('gene2accession: {}'.format(len(gene2accession)))
display(gene2accession.head())

### Loading Transcripts fasta

In [ ]:
TRANSCRIPTOME_FILE = os.path.join(RESULTS, DATASET, 'trinity_assembly', 'Trinity.fasta.gz')
data = []
with gzip.open(TRANSCRIPTOME_FILE, "rt") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        data.append([record.id, len(record.seq)])
trans_len = pandas.DataFrame(data)

In [ ]:
trans_len.head()

### Loading sample list from GCP operations

In [ ]:
samples = pandas.read_csv(os.path.join(RESULTS, DATASET, 'annotation','gcp', 'operations.tsv'), sep='\t')
samples = samples['sample']
bar_length = len(samples)
print('{} fasta files to annotate'.format(len(samples)))

### BlastP

In [ ]:
%%time
THREADS = 25
MIN_BITSCORE = 100

def worker(s):
    """worker function"""    
    dna = os.path.join(RESULTS, DATASET, 'annotation', 'blasts', s, s + '_nocont.fsa.gz')
    prot = os.path.join(RESULTS, DATASET, 'annotation', 'blasts', s, s + '_nocont_transdecoder.fsa.gz')     
    fasta = []
    with gzip.open(dna, "rt") as dna_handle:
        for dna_record in SeqIO.parse(dna_handle, "fasta"):
            if len(dna_record.seq) >= 200:
                fasta.append(dna_record.id)
    df_blast_p = pandas.read_csv(os.path.join(RESULTS, DATASET, 'annotation', 'blasts', s, s + '_nocont_transdecoder_blastp.tsv.gz'), sep='\t', header=None)
    df_blast_p = df_blast_p[df_blast_p[4] >= MIN_BITSCORE]   # Filtering by bitscore bigger than 50
    df_blast_p = df_blast_p[df_blast_p[6].isin(viridiplantae)]
    new = df_blast_p[0].str.split(".", n = 1, expand = True)
    df_blast_p[0] = new[0]
    df_blast_p[7] = new[1]
    df_blast_p = df_blast_p.drop_duplicates(keep='first')  
    df_blast_p = df_blast_p[df_blast_p[0].isin(fasta)]
    
    d = df_blast_p[[0,7]].drop_duplicates(keep='first')
    d = d.groupby(0).count().reset_index()
    trans_with_multiple_proteins = d[d[7] > 1][0].unique()
    df_blast_p_with_multiple_proteins = df_blast_p[df_blast_p[0].isin(trans_with_multiple_proteins)]
    df_blast_p = df_blast_p[~df_blast_p[0].isin(trans_with_multiple_proteins)]
    
    prots = {}     
    with gzip.open(prot, "rt") as prot_handle:          
        for prot_record in SeqIO.parse(prot_handle, "fasta"):
            f = prot_record.id.split('.')
            if f[0] in trans_with_multiple_proteins:
                p = prots.setdefault(f[0], {})   
                d = prot_record.description.split(' ')[1][5:]
                b = df_blast_p_with_multiple_proteins[(df_blast_p_with_multiple_proteins[0] == f[0])&(df_blast_p_with_multiple_proteins[7] == f[1])]                 
                if not p:                    
                    p['d'] = d
                    p['b'] = b
                else:  
                    replace = True if p['d'] != 'complete' and d == 'complete' else False
                    if not replace and p['d'] != 'complete':
                        replace = True if len(p['b']) < len(b) else False                         
                    if replace:     
                        prots[f[0]]['d'] = d
                        prots[f[0]]['b'] = b
    for k, p in prots.items():
        df_blast_p = pandas.concat([df_blast_p, p['b']])
    return {'df': df_blast_p, 'seq': len(fasta)}

# Submitting all samples as jobs
total_seq = 0
blastp = pandas.DataFrame()
p = Pool(processes=THREADS)
data = p.map(worker, [s for s in samples])
for d in data:
    blastp = pandas.concat([blastp, d['df']])
    total_seq += d['seq']
p.close()
print('Total sequences: {}\nBlastP hits: {}'.format(total_seq, len(blastp)))
blastp.to_csv(os.path.join(RESULTS, DATASET, 'annotation', 'blastp.tsv.gz'), header=None, sep='\t', index=None, compression='gzip')
display(blastp.head())

del data

In [ ]:
geneID2ProtGI = gene2accession[['GeneID','protein_accession.version']].drop_duplicates()
geneID2ProtGI = geneID2ProtGI[geneID2ProtGI['protein_accession.version'] != '-']
geneID2ProtGI = gene2go.merge(geneID2ProtGI, on='GeneID')
geneID2ProtGI = geneID2ProtGI[['GO_ID', 'PubMed', '#tax_id', 'protein_accession.version']].drop_duplicates()
new = geneID2ProtGI["protein_accession.version"].str.split(".", n = 1, expand = True) 
geneID2ProtGI['protein_accession'] = new[0]
geneID2ProtGI = geneID2ProtGI[['GO_ID', 'PubMed', '#tax_id', 'protein_accession']].drop_duplicates()

d = blastp[[0,2,7]].drop_duplicates()
new = d[2].str.split(".", n = 1, expand = True) 
d[1] = new[0]
d = d[[0, 1, 7]].drop_duplicates()
trans2go_blastp = d.merge(geneID2ProtGI, left_on=1, right_on='protein_accession')
trans2go_blastp = trans2go_blastp[[0,'GO_ID', 'PubMed', '#tax_id', 7]]
trans2go_blastp = trans2go_blastp.drop_duplicates()
print('\n{} pair transcript GO term from NCBI Gene'.format(len(trans2go_blastp)))
print('{} transcript with GO term from NCBI Gene'.format(len(trans2go_blastp[0].unique())))
trans2go_blastp.to_csv(os.path.join(RESULTS, DATASET, 'blastp_go_gene.tsv.gz'), header=None, sep='\t', index=None, compression='gzip')

trans2go_uniprot = d.merge(idmapping, left_on=1, right_on='protein_accession')
trans2go_uniprot = trans2go_uniprot[[0,'GO', 'UniRef100', 'UniRef90', 'UniRef50', '#tax_id', 7]]
trans2go_uniprot['GO'] = trans2go_uniprot['GO'].str.strip()
trans2go_uniprot = trans2go_uniprot.drop_duplicates()
print('\n{} pair transcript GO term from Uniprot'.format(len(trans2go_uniprot)))
print('{} transcript with GO term from Uniprot'.format(len(trans2go_uniprot[0].unique())))
trans2go_uniprot.to_csv(os.path.join(RESULTS, DATASET, 'blastp_go_uniprot.tsv.gz'), header=None, sep='\t', index=None, compression='gzip')

del d
del geneID2ProtGI

In [ ]:
df1 = trans2go_uniprot[[0,'GO', 7]]
df1 = df1.drop_duplicates()
df2 = trans2go_blastp[[0,'GO_ID', 7]]
df2 = df2.drop_duplicates()
df2 = df2.rename(columns={'GO_ID':'GO'})
df3 = pandas.concat([df1, df2])
df3 = df3.drop_duplicates()
print('\n{} pair transcript GO term'.format(len(df3)))
print('{} transcript with GO term'.format(len(df3[0].unique())))
display(df3.head())

In [ ]:
%%time 

def predecessors(g, O):
    """
    Extract predecessors nodes from an starting node
    :param g: starting node name
    :param O: Graph
    :return: a set with node names
    """
    result = {g}
    for o in O.predecessors(g):
        result.update(predecessors(o, O))
    return result

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def worker(t_list):
    res = pandas.DataFrame()
    for t in t_list:
        df_go = df3[df3[0] == t]    
        l = df_go.GO.unique()    
        to_del = set()
        for i in range(0, len(l) - 1):
            try:
                si = [o for o in predecessors(l[i], go)]
                for j in range(i + 1, len(l)):
                    try:
                        sj = [o for o in predecessors(l[j], go)]        
                        g = go.subgraph(si).subgraph(sj)
                        if len(si) < len(sj) and len(g) == len(si):
                            to_del.add(l[i])
                        elif len(si) > len(sj) and len(g) == len(sj):
                            to_del.add(l[j])
                    except nx.exception.NetworkXError:
                        to_del.add(l[j])
            except nx.exception.NetworkXError:
                to_del.add(l[i])
        if to_del:
            df_go = df_go[~df_go['GO'].isin(to_del)]
        res = pandas.concat([res, df_go])    
    return res

# Submitting all samples as jobs
p = Pool(processes=THREADS)
data = p.map(worker, [d for d in list(chunks(df3[0].unique(), 1500))])
p.close()
blast_go = pandas.DataFrame()
for d in data:
    blast_go = pandas.concat([blast_go, d])

blast_go = blast_go.merge(ec2go, left_on='GO', right_on=1, how='outer')
blast_go = blast_go.rename(columns={'0_x':0,'0_y': 'enzyme'})
blast_go = blast_go[[0,'GO', 7, 'enzyme']]    
blast_go = blast_go[~blast_go[0].isnull()]
blast_go.to_csv(os.path.join(RESULTS, DATASET, 'blastp_go.tsv.gz'), header=None, sep='\t', index=None, compression='gzip') 
display(blast_go.head())

In [ ]:
df4 = blast_go[[0, 'GO']]
df4 = df4.groupby(0).count().reset_index().merge(trans_len, on=0).rename(columns={0:'Transcript', 1:'Length'}).sort_values('Length')

df6 = df4[df4['Length'] <= 5000]
print('{}/{}'.format(len(df6), len(df4)))

fig = plt.figure(figsize=(16,10), constrained_layout=True)

gs = GridSpec(2, 2, figure=fig)

ax0 = fig.add_subplot(gs[0, :])

ax0.bar(df6.Length, height=df6.GO)
ax0.set_title('Barplot transcript length vs no. Go Terms');
ax0.set_ylabel("No of GO Terms")
ax0.set_xlabel("Transcript Length")

ax1 = fig.add_subplot(gs[1, 0])

n, bins, patches = ax1.hist(df6.GO, 100, facecolor='blue', alpha=0.5)
ax1.set_xlabel('No of GO Terms')
ax1.set_ylabel('No of Transcripts')
ax1.set_title('Histogram of GO Terms')

ax2 = fig.add_subplot(gs[1, 1])
n, bins, patches = ax2.hist(df6.Length, 100, facecolor='blue', alpha=0.5)
ax2.set_xlabel('Transcript Length')
ax2.set_ylabel('')
ax2.set_title('Histogram of Transcript Length')